# Lab

In [1]:
from recsys.core.services.io import IOService
import pandas as pd
import numpy as np

In [2]:
FILEPATH = "data/raw/rating.pkl"
USERID = 10

In [3]:

ratings = IOService.read(FILEPATH)
ratings.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 457.8 MB


In [4]:
ratings_user_i = ratings.loc[ratings["userId"] == USERID]


In [5]:
ratings_user_i.info()
ratings_user_i.head()
ratings_user_i.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 922 to 959
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   38 non-null     int64  
 1   movieId  38 non-null     int64  
 2   rating   38 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.2 KB


,userId,movieId,rating
922,10,1,4.0
923,10,11,4.0
924,10,25,4.0
925,10,260,4.0
926,10,356,3.0


(38, 3)

In [6]:
neighbors = ratings
neighbors.columns = ["neighborId", "movieId", "neighbor_rating"]

In [7]:
neighbors.info()
neighbors.head()
neighbors.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 3 columns):
 #   Column           Dtype  
---  ------           -----  
 0   neighborId       int64  
 1   movieId          int64  
 2   neighbor_rating  float64
dtypes: float64(1), int64(2)
memory usage: 457.8 MB


,neighborId,movieId,neighbor_rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


(20000263, 3)

In [9]:
user_neighbor = ratings_user_i.merge(right=neighbors, how="inner", on="movieId").sort_values(by=["neighborId","movieId"])

In [10]:
user_neighbor.info()
user_neighbor.head()
user_neighbor.shape
user_neighbor["neighborId"].nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 906684 entries, 89410 to 806823
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   userId           906684 non-null  int64  
 1   movieId          906684 non-null  int64  
 2   rating           906684 non-null  float64
 3   neighborId       906684 non-null  int64  
 4   neighbor_rating  906684 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 41.5 MB


,userId,movieId,rating,neighborId,neighbor_rating
89410,10,260,4.0,1,4.0
348716,10,1136,4.0,1,3.5
381740,10,1196,4.0,1,4.5
427053,10,1198,4.0,1,4.5
482726,10,1208,4.0,1,3.5


(906684, 5)

125409

In [15]:
weights = user_neighbor.groupby(by=["userId","neighborId"], group_keys=False)[["rating", "neighbor_rating"]].apply(lambda x: (x["rating"].dot(x["neighbor_rating"])/(np.linalg.norm(x["rating"]) * np.linalg.norm(x["neighbor_rating"])))).to_frame().reset_index()
weights.columns = ["userId", "neighborId", "weight"]

In [16]:
weights.info()
weights.head()
weights.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125409 entries, 0 to 125408
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   userId      125409 non-null  int64  
 1   neighborId  125409 non-null  int64  
 2   0           125409 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.9 MB


,userId,neighborId,0
0,10,1,0.992826270602637
1,10,2,1.0
2,10,3,0.983565127906229
3,10,4,1.0
4,10,5,1.0000000000000002


(125409, 3)